### (1) queue包

In [ ]:
import queue

#### (1.1) queue.Queue

##### (1.1.1) 先入先出的队列(FIFO)，拿的是queue最早进来的数据

In [2]:
q = queue.Queue()

for i in range(0, 5):
    q.put(i)

for i in range(0, 5):
    print(q.get())

0
1
2
3
4


##### (1.2) 可以用maxsize来控制队列大小，不设置maxisize时默认为0，此时Queue会不断增长。但是注意，当Queue.put()超过maxsize时，会出现阻塞情况(block)

In [1]:
q = queue.Queue(maxsize=3)

for i in range(0, 5):
    q.put(i)

KeyboardInterrupt: 

#### (1.2) queue.LifoQueue

##### (1.2.1) 后入先出的队列(LIFO)，拿的是queue最晚进来的数据。maxsize设置和Queue相同

In [2]:
q = queue.LifoQueue()

for i in range(0, 5):
    q.put(i)

for i in range(0, 5):
    print(q.get())

4
3
2
1
0


### (2) collections.deque

In [3]:
from collections import deque

#### (2.1) 双向队列，设置了maxlen后可以实现在queue满了的情况下，再append一个数就自动pop一个数

In [7]:
d = deque(maxlen=5)

for i in range(0, 4):
    d.append(i)

print(list(d))

d.append(4)  # 此时队列已经满了

print(list(d))

d.append(5)  # 再append数据，会将前面的数据自动pop，保持queue的定长

print(list(d))

[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]


### (3) 一段用threading包和deque来模拟一个独立产生数据和独立处理数据的多线程例子

In [1]:
from threading import Thread
from collections import deque
import time
import copy
import numpy as np

# 生成一个继承了Thread的多线程的类Processor
class Processor(Thread):

    def __init__(self,  maxlen=None):
        super(Processor, self).__init__()
        self._cache = deque(maxlen=maxlen)
    
    @property  # 装饰器，保证deque的更新只在Processor里面进行(需要再深入学习一下)
    def cache(self):
        return copy.deepcopy(self._cache)

    def append_data(self):
        # 循环地queue.append()数据，模拟一个独立地收数据地进程
        i = 0
        while i <= 100:
            self._cache.append(i)
            i = i + 1
            if i > 100:
                i = 0
            time.sleep(1)

    def run(self):  # 重新定义一下run
        self.append_data()
        

if __name__ == '__main__':
    p = Processor(maxlen=5)
    p.start()
    for n in range(0, 5):
        # 随机地读queue地数据，模拟一个独立地拿数据并进行处理的进程
        time.sleep(10*np.random.rand(1))
        q_list = list(p.cache)
        q_list_len = len(q_list)
        print(f'--get queue data {n+1} times: {q_list}--')
        print(f'lastest data of queue: {q_list[q_list_len-1]}')

--get queue data 1 times: [3, 4, 5, 6, 7]--
lastest data of queue: 7
--get queue data 2 times: [8, 9, 10, 11, 12]--
lastest data of queue: 12
--get queue data 3 times: [10, 11, 12, 13, 14]--
lastest data of queue: 14
--get queue data 4 times: [16, 17, 18, 19, 20]--
lastest data of queue: 20
--get queue data 5 times: [25, 26, 27, 28, 29]--
lastest data of queue: 29
